In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Introduction

In [19]:
# Florence-2 is capable of object detection, captioning and other tasks. Here is a finetuned version of it to detect only person and faces and to infer gender in video

# Check GPU availability

In [2]:
!nvidia-smi

Mon Sep  9 00:15:05 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P0             29W /  250W |       0MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Install required libraries

In [4]:
!pip install -q transformers flash_attn timm
!pip install -q git+https://github.com/roboflow/supervision.git

# Import required dependencies

In [5]:
import torch
import supervision as sv
from tqdm import tqdm
from transformers import (
    AutoModelForCausalLM,
    AutoProcessor,
)

# Load florence-2 and it's preprocessor

In [6]:
CHECKPOINT = "microsoft/Florence-2-base-ft"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AutoModelForCausalLM.from_pretrained(CHECKPOINT, trust_remote_code=True).to(DEVICE)
processor = AutoProcessor.from_pretrained(CHECKPOINT, trust_remote_code=True)

config.json:   0%|          | 0.00/2.43k [00:00<?, ?B/s]

configuration_florence2.py:   0%|          | 0.00/15.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-base-ft:
- configuration_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_florence2.py:   0%|          | 0.00/127k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-base-ft:
- modeling_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

processing_florence2.py:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-base-ft:
- processing_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer_config.json:   0%|          | 0.00/34.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.10M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

# Helper function to focus person and face

In [8]:
def get_refined_response(response):
  count = 0
  index = []
  for label in response['<OD>']['labels']:
    if label=='human face' or label=='face' or label=='person':
      index.append(count)
    count = count + 1
  bboxes = [response['<OD>']['bboxes'][i] for i in index]
  labels = [response['<OD>']['labels'][i] for i in index]
  print(bboxes,labels)
  refined_response = {
    "<OD>": {
        "bboxes": bboxes,
        "labels": labels
    }
  }
  return refined_response

# Helper function to perform person and face detection

In [9]:
def detect_person_face(image,width,height):
  task = "<OD>"
  text = "<OD>"

  inputs = processor(text=text, images=image, return_tensors="pt").to(DEVICE)
  generated_ids = model.generate(
      input_ids=inputs["input_ids"],
      pixel_values=inputs["pixel_values"],
      max_new_tokens=1024,
      num_beams=3
  )
  generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
  response = processor.post_process_generation(generated_text, task=task, image_size=(width,height)) #image_size=(image.width, image.height))
  refined_response = get_refined_response(response)
  detections = sv.Detections.from_lmm(sv.LMM.FLORENCE_2, refined_response, resolution_wh=image.size)
  bounding_box_annotator = sv.BoxAnnotator(color_lookup=sv.ColorLookup.INDEX)
  label_annotator = sv.LabelAnnotator(color_lookup=sv.ColorLookup.INDEX)
  image = bounding_box_annotator.annotate(image, detections)
  image = label_annotator.annotate(image, detections)
  return image

# Helper function to perform gender inference

In [11]:
def detect_gender(image,width,height):
  task = "<CAPTION>"
  text = "<Gender>"

  inputs = processor(text=text, images=image, return_tensors="pt").to(DEVICE)
  generated_ids = model.generate(
    input_ids=inputs["input_ids"],
    pixel_values=inputs["pixel_values"],
    max_new_tokens=1024,
    num_beams=3
  )
  generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
  response = processor.post_process_generation(generated_text, task=task, image_size=(width,height)) #image_size=(image.width, image.height))
  print(response['<CAPTION>'])

# Load video and perform person and face detection

In [12]:
SOURCE_VIDEO_PATH = '/kaggle/input/new-dataset/4k_medium.mp4'
TARGET_VIDEO_PATH = '/kaggle/working//4k_medium.mp4'
generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)
video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

with sv.VideoSink(target_path=TARGET_VIDEO_PATH, video_info=video_info) as sink:
    for frame in tqdm(generator, total=video_info.total_frames):
        frame = detect_person_face(frame,video_info.width,video_info.height)
        sink.write_frame(frame)

  0%|          | 1/451 [00:01<08:47,  1.17s/it]

[[73.02300262451172, 88.74000549316406, 99.28300476074219, 133.02000427246094], [26.76500129699707, 80.46000671386719, 155.4390106201172, 359.46002197265625]] ['human face', 'person']


  0%|          | 2/451 [00:01<04:54,  1.53it/s]

[[72.61900329589844, 89.46000671386719, 99.68700408935547, 133.3800048828125]] ['human face']


  1%|          | 3/451 [00:01<03:38,  2.05it/s]

...

[[37.47100067138672, 106.02000427246094, 55.85300064086914, 131.94000244140625], [142.71299743652344, 124.02000427246094, 161.49900817871094, 146.70001220703125], [79.68900299072266, 127.26000213623047, 94.031005859375, 146.70001220703125], [0.10100000351667404, 91.26000213623047, 131.60301208496094, 343.9800109863281], [117.86700439453125, 112.8600082397461, 188.36500549316406, 322.7400207519531]] ['human face', 'human face', 'human face', 'person', 'person']


100%|█████████▉| 450/451 [03:30<00:00,  1.68it/s]

[[39.08700180053711, 104.22000122070312, 57.06500244140625, 130.13999938964844], [143.72300720214844, 123.66000366210938, 162.10501098632812, 146.3400115966797], [78.8810043334961, 126.9000015258789, 93.02100372314453, 146.70001220703125]] ['human face', 'human face', 'human face']


100%|██████████| 451/451 [03:30<00:00,  2.14it/s]

[[143.31900024414062, 122.94000244140625, 162.3070068359375, 146.3400115966797], [41.10700225830078, 102.7800064086914, 57.46900177001953, 127.9800033569336], [77.66900634765625, 126.54000854492188, 91.80900573730469, 146.3400115966797]] ['human face', 'human face', 'human face']


# Load video and perform gender inference

In [13]:
SOURCE_VIDEO_PATH = '/kaggle/input/new-dataset/4k_medium.mp4'
generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)
video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

for frame in tqdm(generator, total=video_info.total_frames):
    detect_gender(frame,video_info.width,video_info.height)

  0%|          | 2/451 [00:00<01:24,  5.30it/s]

female
female


  1%|          | 4/451 [00:00<01:21,  5.48it/s]

female
female


  1%|▏         | 6/451 [00:01<01:24,  5.28it/s]

...

female
female


100%|█████████▉| 449/451 [01:22<00:00,  5.55it/s]

female
female


100%|██████████| 451/451 [01:23<00:00,  5.42it/s]

female
female


# Conclusion

In [15]:
# Though it struggles to infer gender for some frames, it detects preson and face well.